In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

from sklearn.metrics import *
from copy import deepcopy
import matplotlib.pyplot as plt
import math


from tqdm.auto import tqdm
import seaborn as sns

from src.helpers import *
from src.visualize import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
from torch.utils.data import Dataset
import pickle

import optuna
from src.optuna_functions import * 


device = torch.device('cpu') #suposed to be cuda
dtype = torch.float32

import sys
sys.path.append("c:\\Users\\nerea\\OneDrive\\Documentos\\EPFL_MASTER\\PDM\\Project\\PyalData")
# to change for the actual path where PyalData has been cloned

c:\Users\nerea\anaconda3\envs\sinthlab\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
type = 'stim'

In [3]:
path_to_data = './Data/Processed_Data'
if type == 'baseline':
        baseline_data = os.path.join(path_to_data, 'Tidy_Sansa_11_04.pkl')

        with open(baseline_data, 'rb') as fp:
                df_baseline = pickle.load(fp)


        X_train, y_train, X_val, y_val, X_test,\
        y_test, info_train, info_val, info_test = \
        train_test_split(df_baseline, train_variable = 'both_rates', \
                        target_variable = 'target_pos', num_folds = 5)
elif type == 'stim':
        stim_data = os.path.join(path_to_data, 'Tidy_Sansa_13_04.pkl')
        with open(stim_data, 'rb') as fp:
                df = pickle.load(fp)
        df_stim = df.loc[df.type == 'TONIC'].reset_index()

        X_train, y_train, X_val, y_val, X_test,\
        y_test, info_train, info_val, info_test = \
        train_test_split(df_stim, train_variable = 'both_rates', \
                        target_variable = 'target_pos', num_folds = 5)

# Test one of the folds first
fold_num = 'fold0'
fold = 0

print('We are testing the optimization method on fold ', fold)


X_train = X_train[fold_num]
X_val = X_val[fold_num]
X_test = X_test[fold_num]
y_test = y_test[fold_num]
y_train = y_train[fold_num]
y_val = y_val[fold_num]


# Specify that we want our tensors on the GPU and in float32
device = torch.device('cpu') #suposed to be cuda
dtype = torch.float32
path_to_models = './Models'

num_dim_output = y_train.shape[1]
num_features = X_train.shape[1]

seq_length = 75

# Reshape x_train to match the number of columns in the model's input layer
xx_train = X_train.reshape(X_train.shape[0] // seq_length, seq_length, X_train.shape[1])  
# Reshape y_train to match the number of neurons in the model's output layer
yy_train = y_train.reshape(y_train.shape[0] // seq_length, seq_length, y_train.shape[1])  

xx_val = X_val.reshape(X_val.shape[0] // seq_length, seq_length, X_val.shape[1])  
yy_val = y_val.reshape(y_val.shape[0] // seq_length, seq_length, y_val.shape[1])  

xx_test = X_test.reshape(X_test.shape[0] // seq_length, seq_length, X_test.shape[1])  
yy_test = y_test.reshape(y_test.shape[0] // seq_length, seq_length, y_test.shape[1])  

seed = 42
torch.manual_seed(seed)

Reg = globals().get(Regularizer_LSTM)


Test trials  8
Val trials 7
We are testing the optimization method on fold  0


In [4]:
# Fit the LSTM model
def train_model_optuna(trial):
    
    X = xx_train
    Y = yy_train
    X_val = xx_val
    Y_val = yy_val

    num_epochs= 1000
    early_stop = 5

    model = CausalTemporalLSTM_Optuna(trial, num_features= num_features, 
                out_dims = num_dim_output).to(device)
    
    # Set up the optimizer with the specified learning rate
    #optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer_name = 'Adam'
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log = True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr = lr)

    lr_step_size = 10 # trial.suggest_int('lr_step_size', 5, 15)
    lr_gamma = trial.suggest_float('lr_gamma', 1, 1.5)
    # Set up a learning rate scheduler
    scheduler = lr_scheduler.StepLR(optimizer, 
                                    step_size=lr_step_size, 
                                    gamma=lr_gamma)
        
    # Keep track of the best model's parameters and loss
    best_model_wts = deepcopy(model.state_dict())
    best_loss = 1e8

    # Enable anomaly detection for debugging
    torch.autograd.set_detect_anomaly(True)

    # Track the train and validation loss
    train_losses = []
    val_losses = []
    # Counters for early stopping
    not_increased = 0
    end_train = 0
    
    # Reshape data for the LSTM
    seq_length_LSTM = trial.suggest_int('seq_length_LSTM', 5, 20)
    train_dataset = SequenceDataset(Y,X,seq_length_LSTM)
    val_dataset = SequenceDataset(Y_val,X_val,seq_length_LSTM)


    batch_size_train = trial.suggest_int('batch_size_train', 20, 50)
    batch_size_val = trial.suggest_int('batch_size_val',20, 50)
    loader_train = data.DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
    loader_val = data.DataLoader(val_dataset, batch_size=batch_size_val, shuffle=True)

    # hyperparameter for huber loss
    delta = trial.suggest_int('huber_delta', 5, 10) #was 8 before

    # hyperparameter for regularizer
    alpha = trial.suggest_float('alpha_reg', 1e-7, 1e-3, log = True) # was 1e-5 before
    l1_ratio =trial.suggest_float('l1_ratio_reg', 0.2, 0.8) #was 0.5 before

    # Loop through epochs
    for epoch in np.arange(num_epochs):
        for phase in ['train', 'val']:
            # set model to train/validation as appropriate
            if phase == 'train':
                model.train()
                loader = loader_train
            else:
                model.eval()
                loader = loader_val

            # Initialize variables to track loss and batch size
            running_loss = 0
            running_size = 0        

            # Iterate over batches in the loader
            for X_, y_ in loader:
                #X_ = X_.to('cuda')
                #y_ = y_.to('cuda')
                if phase == "train":
                    with torch.set_grad_enabled(True):
                        optimizer.zero_grad()

                        output_t = model(X_)
                        output_t = torch.squeeze(output_t)


                        loss_t = huber_loss(output_t, y_, delta)
                        
                        # Add regularization to the loss in the training phase
                        loss_t += Regularizer_LSTM(model, l1_ratio, alpha)

                        # Compute gradients and perform an optimization step
                        loss_t.backward()
                        optimizer.step()
                else:
                    # just compute the loss in validation phase
                    output_t = model(X_)
                    output_t = torch.squeeze(output_t)

                    loss_t = huber_loss(output_t, y_, delta)

                # Ensure the loss is finite
                assert torch.isfinite(loss_t)
                running_loss += loss_t.item()
                running_size += 1

            # compute the train/validation loss and update the best
            # model parameters if this is the lowest validation loss yet
            running_loss /= running_size
            if phase == "train":
                train_losses.append(running_loss)
            else:
                val_losses.append(running_loss)
                
                # Update best model parameters if validation loss improves
                if running_loss < best_loss:
                    best_loss = running_loss
                    best_model_wts = deepcopy(model.state_dict())
                    not_increased = 0
                else:
                    # Perform early stopping if validation loss doesn't improve
                    if epoch > 10:
                        not_increased += 1
                        # print('Not increased : {}/5'.format(not_increased))
                        if not_increased == early_stop:
                            print('Decrease LR')
                            for g in optimizer.param_groups:
                                g['lr'] = g['lr'] / 2
                            not_increased = 0
                            end_train += 1
                        
                        if end_train == 2:
                            model.load_state_dict(best_model_wts)
                            y_true_val, y_pred_val = reshape_to_eval(xx_val,yy_val, model)
                            ev_val = explained_variance_score(y_true_val, y_pred_val)
                            return ev_val  # here change to not return a list but a single value for the trial to analyze
                        

        # Update learning rate with the scheduler
        scheduler.step()
        print("Epoch {:03} Train {:.4f} Val {:.4f}".format(epoch, train_losses[-1], val_losses[-1]))


        y_true_val_epoch, y_pred_val_epoch = reshape_to_eval(xx_val,yy_val, model)
        ev_val_epoch = explained_variance_score(y_true_val_epoch, y_pred_val_epoch)
        trial.report(ev_val_epoch, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    y_true_val, y_pred_val = reshape_to_eval(xx_val,yy_val, model)
    ev_val = explained_variance_score(y_true_val, y_pred_val)

    return ev_val


In [5]:
def reshape_to_eval(x,y, model):
    to_t_eval =  lambda array: torch.tensor(array, device='cpu', dtype=dtype)  
    x = to_t_eval(x) 
    y = to_t_eval(y)
    y_pred = model(x)
    y_array = y.detach().cpu().numpy()
    y_pred_array = y_pred.detach().cpu().numpy()

    # Reshape tensors to 2D arrays (flatten the batch and sequence dimensions)
    y_pred_2D = y_pred_array.reshape(-1, y_pred_array.shape[-1])
    y_true_2D = y_array.reshape(-1, y_array.shape[-1])
    
    return y_true_2D, y_pred_2D

In [6]:
study = optuna.create_study(direction="maximize")

# Set pruning options
study.pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5)

study.optimize(train_model_optuna, n_trials=30)

importance_scores = optuna.importance.get_param_importances(study)

# Print importance scores
for param, score in importance_scores.items():
    print(f"{param}: {score}")

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print('Study statistics: ')
print("Number of finished trials: ", len(study.trials))
print("Number of pruned trials: ", len(pruned_trials))
print("Number of complete trials: ", len(complete_trials))

print('Best trial: ')
trial = study.best_trial

print("Loss: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-04-10 16:31:58,891] A new study created in memory with name: no-name-a492c491-9ce9-482e-b3a0-051993dbd14f


Epoch 000 Train 0.3639 Val 0.0248
Epoch 001 Train 0.4548 Val 0.0249
Epoch 002 Train 0.5190 Val 0.0234
Epoch 003 Train 0.5814 Val 0.0231
Epoch 004 Train 0.6541 Val 0.0231
Epoch 005 Train 0.7178 Val 0.0219
Epoch 006 Train 0.7777 Val 0.0258
Epoch 007 Train 0.8301 Val 0.0297
Epoch 008 Train 0.8703 Val 0.0277
Epoch 009 Train 0.9047 Val 0.0264
Epoch 010 Train 0.9478 Val 0.0258
Epoch 011 Train 0.9896 Val 0.0286
Epoch 012 Train 1.0334 Val 0.0271
Epoch 013 Train 1.0808 Val 0.0283
Epoch 014 Train 1.1284 Val 0.0282
Decrease LR
Epoch 015 Train 1.1727 Val 0.0275
Epoch 016 Train 1.2015 Val 0.0269
Epoch 017 Train 1.2113 Val 0.0296
Epoch 018 Train 1.2192 Val 0.0263
Epoch 019 Train 1.2276 Val 0.0266


[I 2024-04-10 16:34:05,755] Trial 0 finished with value: 0.4306870897610982 and parameters: {'n_hidden_units': 59, 'num_layers': 5, 'input_size_LSTM': 36, 'dropout_1': 0.8619303502433577, 'dropout_2': 0.07033144188011413, 'lr': 0.006901422960689535, 'lr_gamma': 1.103251854957949, 'seq_length_LSTM': 9, 'batch_size_train': 25, 'batch_size_val': 20, 'huber_delta': 5, 'alpha_reg': 0.0008225967018626142, 'l1_ratio_reg': 0.3949581248343751}. Best is trial 0 with value: 0.4306870897610982.


Decrease LR
Epoch 000 Train 0.0367 Val 0.0348
Epoch 001 Train 0.0360 Val 0.0346
Epoch 002 Train 0.0353 Val 0.0343
Epoch 003 Train 0.0342 Val 0.0334
Epoch 004 Train 0.0324 Val 0.0318
Epoch 005 Train 0.0300 Val 0.0296
Epoch 006 Train 0.0279 Val 0.0275
Epoch 007 Train 0.0263 Val 0.0263
Epoch 008 Train 0.0251 Val 0.0253
Epoch 009 Train 0.0242 Val 0.0249
Epoch 010 Train 0.0234 Val 0.0243
Epoch 011 Train 0.0226 Val 0.0239
Epoch 012 Train 0.0220 Val 0.0237
Epoch 013 Train 0.0213 Val 0.0234
Epoch 014 Train 0.0209 Val 0.0232
Epoch 015 Train 0.0204 Val 0.0232
Epoch 016 Train 0.0200 Val 0.0231
Epoch 017 Train 0.0194 Val 0.0230
Epoch 018 Train 0.0191 Val 0.0228
Epoch 019 Train 0.0186 Val 0.0230
Epoch 020 Train 0.0181 Val 0.0228
Epoch 021 Train 0.0176 Val 0.0231
Epoch 022 Train 0.0174 Val 0.0229
Decrease LR
Epoch 023 Train 0.0171 Val 0.0231
Epoch 024 Train 0.0169 Val 0.0226
Epoch 025 Train 0.0166 Val 0.0226
Epoch 026 Train 0.0165 Val 0.0226
Epoch 027 Train 0.0165 Val 0.0224
Epoch 028 Train 0.0163 V

[I 2024-04-10 16:36:08,814] Trial 1 finished with value: 0.477204163869222 and parameters: {'n_hidden_units': 24, 'num_layers': 3, 'input_size_LSTM': 41, 'dropout_1': 0.8520513042122673, 'dropout_2': 0.8198070091307547, 'lr': 0.00011289041103035705, 'lr_gamma': 1.0226660667782599, 'seq_length_LSTM': 14, 'batch_size_train': 50, 'batch_size_val': 24, 'huber_delta': 10, 'alpha_reg': 7.313490805237113e-07, 'l1_ratio_reg': 0.774533467013518}. Best is trial 1 with value: 0.477204163869222.


...

Epoch 008 Train 0.0089 Val 0.0253
Epoch 000 Train 0.0883 Val 0.0330
Epoch 001 Train 0.0764 Val 0.0229
Epoch 002 Train 0.0734 Val 0.0244
Epoch 003 Train 0.0724 Val 0.0238
Epoch 004 Train 0.0727 Val 0.0257
Epoch 005 Train 0.0733 Val 0.0250


[I 2024-04-10 17:09:38,600] Trial 28 pruned. 


Epoch 006 Train 0.0739 Val 0.0261
Epoch 000 Train 0.0366 Val 0.0217
Epoch 001 Train 0.0342 Val 0.0245
Epoch 002 Train 0.0358 Val 0.0211
Epoch 003 Train 0.0376 Val 0.0209
Epoch 004 Train 0.0390 Val 0.0190
Epoch 005 Train 0.0406 Val 0.0206
Epoch 006 Train 0.0422 Val 0.0218
Epoch 007 Train 0.0435 Val 0.0225
Epoch 008 Train 0.0450 Val 0.0231
Epoch 009 Train 0.0473 Val 0.0226
Epoch 010 Train 0.0498 Val 0.0236
Epoch 011 Train 0.0521 Val 0.0220
Epoch 012 Train 0.0541 Val 0.0202
Epoch 013 Train 0.0563 Val 0.0216
Epoch 014 Train 0.0587 Val 0.0272
Decrease LR
Epoch 015 Train 0.0609 Val 0.0248
Epoch 016 Train 0.0618 Val 0.0225
Epoch 017 Train 0.0620 Val 0.0246
Epoch 018 Train 0.0622 Val 0.0232
Epoch 019 Train 0.0624 Val 0.0229


[I 2024-04-10 17:11:42,541] Trial 29 finished with value: 0.6101757884025574 and parameters: {'n_hidden_units': 48, 'num_layers': 5, 'input_size_LSTM': 28, 'dropout_1': 0.6301115710125259, 'dropout_2': 0.28410657229673986, 'lr': 0.007054410271015403, 'lr_gamma': 1.082977632333134, 'seq_length_LSTM': 14, 'batch_size_train': 25, 'batch_size_val': 20, 'huber_delta': 5, 'alpha_reg': 6.200631424405033e-05, 'l1_ratio_reg': 0.37666505231993336}. Best is trial 14 with value: 0.6294087171554565.


Decrease LR
seq_length_LSTM: 0.28027618437561724
dropout_1: 0.20743737365660045
dropout_2: 0.11288424008265828
n_hidden_units: 0.11235759649298317
input_size_LSTM: 0.10130934845745654
batch_size_val: 0.03696156263780728
lr_gamma: 0.03207683000616677
num_layers: 0.029609800630826463
l1_ratio_reg: 0.02782169689956927
lr: 0.02470460686785145
alpha_reg: 0.020834567530554685
huber_delta: 0.009694289294782403
batch_size_train: 0.004031903067125941
Study statistics: 
Number of finished trials:  30
Number of pruned trials:  9
Number of complete trials:  21
Best trial: 
Loss: 0.6294087171554565
Best hyperparameters: {'n_hidden_units': 32, 'num_layers': 5, 'input_size_LSTM': 31, 'dropout_1': 0.45353387311567006, 'dropout_2': 0.43469775095920277, 'lr': 0.0017186636685695396, 'lr_gamma': 1.4869841187864843, 'seq_length_LSTM': 17, 'batch_size_train': 33, 'batch_size_val': 34, 'huber_delta': 7, 'alpha_reg': 6.5952774295107e-06, 'l1_ratio_reg': 0.38894691744111975}


In [12]:
import joblib
import json
# Retrieve the best trial
best_trial = study.best_trial

# Get the best hyperparameters
best_params = best_trial.params

# Save the best parameters
with open('stim_0411_best_params.json', 'w') as f:
    json.dump(best_params, f)

### Plotting the study

In [7]:
optuna.visualization.plot_optimization_history(study)

In [8]:
#Plotting the accuracies for each hyperparameter for each trial.
    
optuna.visualization.plot_slice(study)

In [9]:
# Plotting the optimization history of the study.

optuna.visualization.plot_optimization_history(study)

#Plotting the accuracies for each hyperparameter for each trial.
    
optuna.visualization.plot_slice(study)

# Plotting the accuracy surface for the hyperparameters involved in the random forest model.

optuna.visualization.plot_contour(study, params=["seq_length_LSTM", "n_hidden_units"]) 